In [28]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, RandomSampler
from torch.optim import Adam
from torchvision import datasets
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from PIL import Image
import torch.nn as nn
from torchvision import transforms
from sklearn.metrics import accuracy_score



if torch.cuda.is_available():        
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("device:", device)

device: cpu


# 1. Prepare The Image Data (As Before)

In [29]:
def extract_path(tup):
    return tup[0]

train = datasets.ImageFolder("data/cat-and-dog/training_set")
train_imgs = pd.Series(train.imgs, name="path").apply(func=extract_path)
train_targets = pd.Series(train.targets, name="label")
train_merged = pd.concat([train_imgs, train_targets], axis=1)
train, val = train_test_split(train_merged, test_size=0.3, random_state=0)

test = datasets.ImageFolder("data/cat-and-dog/test_set")
test_imgs = pd.Series(test.imgs, name="path").apply(func=extract_path)
test_targets = pd.Series(test.targets, name="label")
test_merged = pd.concat([test_imgs, test_targets], axis=1)

Take a look at the target distribution:

In [30]:
train["label"].value_counts()/len(train)

1    0.500803
0    0.499197
Name: label, dtype: float64

In [31]:
val["label"].value_counts()/len(val)

0    0.500833
1    0.499167
Name: label, dtype: float64

In [32]:
class CustomDataset(Dataset):
    """
    A custom image dataset.
    """
    def __init__(self, data, transform_pipe, device):
        self.data = data
        self.transform_pipe = transform_pipe
        self.device = device
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        observation = self.data.iloc[index, :]
        image = Image.open(observation["path"]).convert("RGB")
        image = self.transform_pipe(image).to(self.device)
        if observation["label"] == 0:
            label = torch.zeros(size=(1,1)).to(self.device)
        else:
            label = torch.ones(size=(1,1)).to(self.device)
        return [image, label]

# 2. Batch Normalization Introduction

To understand the concept of [Batch Normalization](http://proceedings.mlr.press/v37/ioffe15.pdf), we should step back and think about Normalization (often referred to as Standardization) itself.
Normalization...

* ...refers to adjusting numerical values to be on a common scale.

* ...allows our model to be more generalizing

Batch Normalization utilizes a Normalization technique similar to [Standardization](https://www.kaggle.com/milankalkenings/comprehensive-tutorial-feature-engineering).
Standardization subtracts the mean of the data and devides it by its standard deviation. The normalization technique used by Batch Normalization subtracts the mean of the data and devides it by the variance of the data

## How does Batch Normalization work?

Whenever we train a deep learning model, the parameters in each layer are adapted over the training iterations. In iteration $\mathscr{i}$, the parameters of layer $\mathscr{l}$ will be adapted to generate optimal outputs for the given input. However, training the model will change the input of layer $\mathscr{l}$, since the parameters of layer $\mathscr{l-1}$ will be changed as well. Thus, the parameters of layer $\mathscr{l}$ are already slightly outdated in iteration $\mathscr{i}+1$. The aforementioned change in the input distribution is reffered to as [Internal Covariance Shift](http://proceedings.mlr.press/v37/ioffe15.pdf) (note: the more general term [Covariance Shift](https://www.youtube.com/watch?v=nUUqwaxLnWs&t=64s) refers to changes in our data that require retraining the model).
Batch Normalization evades this issue by performing Standardization per batch at the given position in the network using the following formulas using $N$ many inputs:

$\mu_{batch}=\frac{1}{N}\sum^N_nx_n$

$\sigma_{batch}^2=\frac{1}{N}\sum^N_n(x_n-\mu_{batch})^2$

$\hat x_n=\frac{x_n-\mu_{batch}}{\sqrt{\sigma^2 + \epsilon}}$

return: $\gamma \hat x_n + \beta$

Note: $\gamma \hat x_n + \beta$ is solely used to allow the network to learn its optimal mean $\beta$ and deviation $\gamma$ on its own. Both parameters are learned during the training process.




***
For those of you who already made some experiences with Machine Learning Ensembles:

Interpreting each node in our neural network as an **isolated** weak learner in a stacking ensemble, we can easily derive the idea of batch normalization.
The nodes (weak learners) at layer $\mathscr{l}$ don't know wether the outputs of layer $\mathscr{l-1}$ are the actual input data (in case $\mathscr{l-1} = 0$) or already processed data from any hidden layer. 
Since we know that normalization in many cases improves the performance of machine learning models by allowing them to be more generalizing, we should enable all of these weak learners to benefit from that. 
Thus, normalizing the data for each (or at least for some to have less time complexity) non-input layer seems to be a good idea. 
Since we only feed batches into our neural network, we have no idea about the actual normalization parameters across the whole dataset and instead can only predict them on the given batch - we arrived at the idea of batch normalization.

# Benefits of Batch Normalization

The [Batch Normalization Paper](http://proceedings.mlr.press/v37/ioffe15.pdf) lists a number of practical beneftis. 

* Batch Normalization allows layers to learn slightly more independently from other layers.

* Batch Normalization reduces the impact of the data scale on the gradients, making them more dependend on the actual patterns in the input and less on single large values.

* Each layer can assume its input to be equally distributed on train and test

* The model will be less sensitive to parameter initialization.

* The resulting gradients are more balanced and thus less likely to explode or vanish. Exploding and vanishing gradients are caused by backpropagation itself. To reach weights in the early layers of our model, we need to apply the chain rule all the way down to the respective weights. Let me give you an extreme example for each case: If the values in the chain rule are all bigger than 1, the respective gradient might be really big (explode). If the values in the chain rule are all smaller than 1, the respective gradient might be very small. Since the gradients are directly used in the parameter updates, they might lead to way too big (exploding), or way too small updates during the training process. The earlier the weights are used in the neural network, the more are they at risk to suffer from the aforementioned issues.

* The model can handle higher learning rates (because Batch Normalization evades vanishing and exploding gradient).

* Batch Normalization prevents saturable nonlinearity functions to saturate (not that important for us but beneficial in very complex architectures that rely on specific nonlinearities)

# 4. Implementing Batch Normalization

Note: Batch Normalization can be applied to image data as well. Therefore we solely have to use `BatchNorm2d` and provide the number of image channels.

In [33]:
class CNNClassifier(nn.Module):
    """
    A CNN-based classifier.
    """

    def __init__(self, conv_ch1, conv_ch2, linear_size, kernel_size, pooling_size, linear_input_size=None):
        """
        Constructor.

        :param int conv_ch1: number of output channels of the first convolutional layer
        :param int conv_ch2: number of output channels of the second convolutional layer
        :param int linear_size: number of outputs the second linear layer expects from the first linear layer
        :param int kernel_size: width and height of each convolutional kernel in the model
        :param int pooling_size: size of the pooling window
        :param int linear_input_size: number of outputs the first linear layer expects from the convolutional
        part.
        """
        super(CNNClassifier, self).__init__()
        self.linear_input_size = linear_input_size
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(pooling_size)

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=conv_ch1, kernel_size=kernel_size)
        
        self.batch_norm1 = nn.BatchNorm2d(num_features=conv_ch1)
        self.conv2 = nn.Conv2d(in_channels=conv_ch1, out_channels=conv_ch2, kernel_size=kernel_size)
        
        if linear_input_size:  # evaluates as False if linear_input_size is None
            self.batch_norm2 = nn.BatchNorm1d(num_features=linear_input_size)
            self.linear1 = nn.Linear(in_features=linear_input_size, out_features=linear_size)
            self.batch_norm3 = nn.BatchNorm1d(num_features=linear_size)
            self.linear2 = nn.Linear(in_features=linear_size, out_features=1)
            self.batch_norm4 = nn.BatchNorm1d(num_features=1)
            self.sigmoid = nn.Sigmoid()

    def conv_part(self, x):
        """
        Calculates the convolutional part of the forward pass.

        :param torch.Tensor x: input data
        :return: input representations after the convolutional part.
        """
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.batch_norm1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        return x

    def scalars_after_conv(self, x):
        """
        Calculates how many scalars, i.e. tensors of rank 0 are in the output of the convolutional component.

        :param torch.Tensor x: one batch of input/observations
        :return: the number of rank 0 tensors in the output of the convolutional component
        """
        x = self.conv_part(x)
        size = x.size()  # batch_size x channel_size x width x height
        n_channels = size[1]
        width = size[2]
        height = size[3]
        return n_channels * width * height

    def forward(self, x):
        """
        performs the forward pass.

        :param  torch.Tensor x: the input/observations per batch
        :return: the prediction of the whole batch
        """
        x = self.conv_part(x)

        if self.linear_input_size:
            x = x.view(-1, self.linear_input_size)  # flatten out for the linear layers
            
            x = self.batch_norm2(x)
            x = self.linear1(x)
            
            x = self.batch_norm3(x)
            x = self.linear2(x)
            
            x = self.batch_norm4(x)
        return self.sigmoid(x)

## Image Normalization
Now that we created a batch-normalized neural network, we should normalize the image data itself as well. We could either do this by aplying batch normalization before the first hidden layer, or use `transforms.Normalize` in our data preprocessing pipeline. I will stick to the latter in order to normalize over the whole dataset. 

In [34]:
def find_mean_std(loader):
    """
    Calculates the averaged mean and std for all channels 
    of a given RGB-image dataset.
    """
    mean_ch1 = 0
    mean_ch2 = 0
    mean_ch3 = 0
    std_ch1 = 0
    std_ch2 = 0
    std_ch3 = 0


    for image in loader:
        means = torch.mean(input=image[0], dim=[2, 3])[0] # mean for each channel
        mean_ch1 += means[0].item()
        mean_ch2 += means[1].item()
        mean_ch3 += means[2].item()

        stds = torch.std(input=image[0], dim=[2, 3])[0] # std for each channel
        std_ch1 += stds[0].item()
        std_ch2 += stds[1].item()
        std_ch3 += stds[2].item()
        
    mean_ch1 /= len(loader)
    mean_ch2 /= len(loader)
    mean_ch3 /= len(loader)
    std_ch1 /= len(loader)
    std_ch2 /= len(loader)
    std_ch3 /= len(loader)
    return {"mean": [mean_ch1, mean_ch2, mean_ch3], "std": [std_ch1, std_ch2, std_ch3]}

In [35]:
# determine mean and std of the training data
example_pipe = transforms.Compose([transforms.RandomCrop(size=[256, 256], pad_if_needed=True),
                                   transforms.ToTensor()])

example_dataset = CustomDataset(data=train, transform_pipe=example_pipe, device=device)
example_loader = DataLoader(example_dataset, 
                            batch_size=1, 
                            sampler=RandomSampler(data_source=example_dataset))

stats = find_mean_std(loader=example_loader)

# define the datasets accordingly
transform_pipe = transforms.Compose([transforms.RandomCrop(size=[256, 256], pad_if_needed=True),
                                     transforms.ToTensor(), 
                                     transforms.Normalize(mean=stats["mean"], std=stats["std"])])

train_dataset = CustomDataset(data=train, transform_pipe=transform_pipe, device=device)
val_dataset = CustomDataset(data=val, transform_pipe=transform_pipe, device=device)
test_dataset = CustomDataset(data=test, transform_pipe=transform_pipe, device=device)

## Training
Using Gradient Accumulation as mentioned in the last part of this series.
Consider we have a GPU capable to load 8 images at a time only, but we want to calculate gradients based on a pseudo batch of 16 images per step:

In [36]:
# Parameters
# accumulate 4 4-batches to one 16-pseudo-batch
combine = 4
batch_size = 4

n_epochs = 5
lr = 0.01  # note: large learning rate
conv_ch1 = 8
conv_ch2 = 4
linear_size = 16
kernel_size = 3
pooling_size = 4
train_loader = DataLoader(train_dataset, 
                          batch_size=batch_size, 
                          sampler=RandomSampler(data_source=train_dataset))
val_loader = DataLoader(val_dataset, 
                        batch_size=batch_size, 
                        sampler=RandomSampler(data_source=val_dataset))

# to automatically determine the first linear size:
example_batch = next(iter(train_loader))
example_x = example_batch[0]
example_model = CNNClassifier(conv_ch1=conv_ch1, 
                              conv_ch2=conv_ch2, 
                              linear_size=linear_size, 
                              kernel_size=kernel_size, 
                              pooling_size=pooling_size).to(device)
linear_input_size = example_model.scalars_after_conv(x=example_x)

model = CNNClassifier(conv_ch1=conv_ch1, 
                      conv_ch2=conv_ch2, 
                      linear_size=linear_size, 
                      kernel_size=kernel_size, 
                      pooling_size=pooling_size, 
                      linear_input_size=linear_input_size).to(device)
optimizer = Adam(model.parameters())
loss_func = nn.BCELoss()

In [37]:
for epoch in range(n_epochs):
    print("=== Epoch", epoch+1, "===")
    acc_train = 0
    acc_val = 0
    # train
    model.train()  # train mode
    for i, batch in enumerate(train_loader):
        x = batch[0]
        y = batch[1].view(-1, 1)
        
        probas = model(x) # perform forward
        loss = loss_func(probas, y) # calculate the loss
        loss /= combine
        loss.backward() # calculate gradients
        
        if ((i+1) % combine == 0):
            optimizer.step() # update weights
            optimizer.zero_grad() # clear the gradient
            
    model.eval()  # evaluation mode
    # acc on train, interesting if you want to
    # check for overfitting
    for batch in train_loader:
        x = batch[0]
        y = batch[1].view(-1, 1)
        with torch.no_grad():
            probas = model(x)
        pred = np.round(probas.cpu().numpy())
        acc_train += accuracy_score(y_true=y.cpu().numpy(), y_pred=pred)
    acc_train /= len(train_loader)
    print("Train Accuracy:", acc_train)
    
    # acc on val
    for batch in val_loader:
        x = batch[0]
        y = batch[1].view(-1, 1)
        with torch.no_grad():
            probas = model(x)
        pred = np.round(probas.cpu().numpy())
        acc_val += accuracy_score(y_true=y.cpu().numpy(), y_pred=pred)
        
    
    acc_val /= len(val_loader)
    print("Val Accuracy:", acc_val, "\n")

=== Epoch 1 ===
Train Accuracy: 0.6045086842731382
Val Accuracy: 0.598585690515807 

=== Epoch 2 ===
Train Accuracy: 0.6109326671425173
Val Accuracy: 0.598585690515807 

=== Epoch 3 ===
Train Accuracy: 0.6290149892933619
Val Accuracy: 0.6335274542429284 

=== Epoch 4 ===
Train Accuracy: 0.6345467523197716
Val Accuracy: 0.6372712146422629 

=== Epoch 5 ===
Train Accuracy: 0.6344872709969069
Val Accuracy: 0.6397670549084858 

